In [4]:
from mst_algorithm import MST
from mst_graphs_local import NetworkXGraph
import networkx as nx
import random

In [5]:
def assign_edge_weights(graph, max_weight):
    for (u, v) in graph.edges():
        graph.edges[u, v]['weight'] = random.randint(0, max_weight)
    return graph

In [6]:
size = 1000
degree = 4
max_weight = 10
max_read = 300

def print_constants():
    output = "Size: "+str(size)+"\n"+"Degree: "+str(degree)+"\n"+"Max Weight: "+str(max_weight)+"\n"+"Max Read: "+str(max_read)
    print(output)
 

In [15]:
graph = nx.generators.random_regular_graph(degree, size)
graph = assign_edge_weights(graph,max_weight)

approx = MST(
    NetworkXGraph(graph, size, max_weight, max_read)
).approx_weight()

print("Our MST weight approx. for random graph:", approx)
print_constants()

Our MST weight approx. for random graph: 2404
Size: 1000
Degree: 4
Max Weight: 10
Max Read: 300


In [11]:
graph = nx.generators.paley_graph(size)
graph = assign_edge_weights(graph,max_weight)
approx = MST(
    NetworkXGraph(graph, size, max_weight, max_read)
).approx_weight()

print("Our MST weight approx. for expanders graph:", approx)
print_constants()

Our MST weight approx. for expanders graph: 999
Size: 1000
Degree: 4
Max Weight: 10
Max Read: 300


In [12]:
graph = nx.generators.complete_graph(size)
graph = assign_edge_weights(graph,max_weight)
approx = MST(
    NetworkXGraph(graph, size, max_weight, max_read)
).approx_weight()

print("Our MST weight approx. for complete graph:", approx)
print_constants()

Our MST weight approx. for complete graph: 999
Size: 1000
Degree: 4
Max Weight: 10
Max Read: 300


In [13]:
graph = nx.generators.cycle_graph(size)
graph = assign_edge_weights(graph,max_weight)
approx = MST(
    NetworkXGraph(graph, size, max_weight, max_read)
).approx_weight()

print("Our MST weight approx. for cycle graph:", approx)
print_constants()


Our MST weight approx. for expanders graph: 5489
Size: 1000
Degree: 4
Max Weight: 10
Max Read: 300
